In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：用于在线预测的自定义训练表回归模型，并具有解释性

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_online_explain_get_metadata.ipynb">
      在Google Cloud笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK来训练和部署一个自定义的表格回归模型，用于在线预测并提供解释。

数据集

本教程使用的数据集是[Boston Housing Prices数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本内置于TensorFlow中。训练好的模型预测房屋的中位价格，单位为1K美元。

### 目标

在本教程中，您将使用 Vertex SDK 在 Google 预建的 Docker 容器中从 Python 脚本创建自定义模型，然后通过发送数据对部署的模型进行预测及解释。您也可以选择使用 `gcloud` 命令行工具或云控制台在线创建自定义模型。

执行的步骤包括：

- 创建用于训练模型的 Vertex 自定义作业。
- 训练一个 TensorFlow 模型。
- 检索并加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为 Vertex `Model` 资源。
- 部署`Model`资源到服务`Endpoint`资源。
- 进行带解释的预测。
- 卸载`Model`资源。

成本

本教程使用Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google云笔记本，则您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简化的指令：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端中的命令行上运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端中的命令行上运行`jupyter notebook`。

6. 在Jupyter Notebook面板中打开此笔记本。

## 安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

本教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您将获得$300的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行作为shell命令运行，并插入以`$`为前缀的Python变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于整个笔记本其余部分的操作。以下是支持Vertex AI的地区。我们建议您选择最靠近您的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能会使用共享测试帐户或项目。为避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###认证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，则您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，请运行下面的单元格，并在提示时按照说明进行oAuth身份验证。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，并选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，并选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境中。

将您的服务帐户密钥路径输入为下面单元格中的GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论在哪个笔记本环境下都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源相关的所有数据跨会话保留的位置。

请在下方设置您的云存储桶的名称。桶的名称必须在包括组织以外的所有谷歌云项目中是全局唯一的。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查 Cloud Storage 存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Vertex SDK以便在您的项目和对应的存储桶中使用Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU的容器映像以及分配给虚拟机实例的GPU数量。例如，要使用一个支持GPU的容器映像，每个虚拟机分配4个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则指定 `(None, None)` 以使用一个容器映像在CPU上运行。

在[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)了解更多有关您地区的硬件加速器支持信息。

*注意*：在2.3版本之前的TF版本中，GPU支持可能无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已经修复--这是由服务功能中生成的静态图操作导致的。如果在您自己的自定义模型中遇到此问题，请使用带有GPU支持的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建Docker容器映像。

有关最新列表，请参见训练的预构建容器。

有关最新列表，请参见预测的预构建容器。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下内容不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的自定义模型并为波士顿住房进行训练。

### 检查培训包

#### 包布局

在开始培训之前，您将查看Python包如何为自定义培训任务进行组装。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的指令。

文件`trainer/task.py`是用于执行自定义培训任务的Python脚本。*注意*，当我们在工作池规范中引用它时，我们用点替换目录斜杠(`trainer.task`)，并删除文件后缀(`.py`)。

#### 包组装

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细解释，只是让您浏览一下。总结如下：

- 从命令行 (`--model_dir`) 或者如果未指定，则从环境变量 `AIP_MODEL_DIR` 获取保存模型结果的目录。
- 从 TF.Keras 内置数据集中加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 用由 `args.epochs` 指定的周期训练模型（`fit()`）。
- 将训练好的模型（`save(args.model_dir)`）保存到指定的模型目录。
- 将每个特征的最大值保存到指定的参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，和 2）运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要以下参数：

- `display_name`：自定义训练任务的人类可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的包要求（如 pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="boston_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的命令行参数

现在定义您自定义训练容器的命令行参数：

- `args`：传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
      - 直接：您将 Cloud Storage 位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或者
      - 间接：服务将 Cloud Storage 位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业说明中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每个时代的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

运行自定义训练作业

接下来，您可以通过调用`run`方法来运行自定义作业，启动训练作业，并提供以下参数：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：要用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `base_output_dir`：写入模型工件的云存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储桶中。现在从云存储桶中加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，可以使用 TF.Keras 中的 `model.load_model()` 方法，将保存模型的云存储路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们来看看模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`加载波士顿房屋测试（留置）数据，使用`load_data()`方法。这将返回一个由两个元素组成的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是由两个元素组成的元组：特征数据和相应的标签（自住房房屋的中位数价值）。

您不需要训练数据，因此我们加载为`(_, _)`。

在您可以将数据进行评估之前，您需要预处理它：

`x_test`：
1. 将每列数据标准化（重新缩放），方法是将每个值除以该列的最大值。这将用32位浮点数替换每个单个值，使它们介于0和1之间。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

现在评估定制工作中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

获取服务功能签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要了解服务功能的输入层名称 -- 在之后进行预测请求时会用到。

您还需要知道服务功能的输入和输出层的名称，以构建解释元数据 -- 这将在后续讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 解释规范

要在进行预测时获取解释，您必须启用解释功能，并在将自定义模型上传到 Vertex“Model”资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于解释您的模型的可解释性算法的规范。您可以在以下选项之间进行选择：
  - Shapley - *注意*，不推荐用于图像数据--可能需要很长时间
  - XRAI
  - 综合梯度
- `metadata`：这是算法应用于您的自定义模型的规范。

#### 解释参数

让我们首先深入了解可解释性算法的设置。

#### Shapley

将对结果归因于每个特征，并考虑特征的不同排列。这种方法提供了确切 Shapley 值的采样近似。

使用案例：
  - 表格数据的分类和回归。

参数：

- `path_count`：这是算法处理的特征路径数量。准确的 Shapley 值近似需要 M! 条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非常数量的特征，这都是计算成本过高。您可以将路径数量减少到 M * `path_count`。

#### 综合梯度

一种基于梯度的方法，可以有效地计算特征归因，具有与 Shapley 值相同的公理属性。

使用案例：
  - 表格数据的分类和回归。
  - 图像数据的分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域以创建显着性图，突出显示图像的相关区域而不是像素。

使用案例：

  - 图像数据的分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

在下一个代码单元中，将变量 `XAI` 设置为您将在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.explain.ExplanationParameters(PARAMETERS)

#### 说明元数据

让我们首先深入了解说明元数据，它包括：

- `outputs`：一个标量值，用于标记输出 -- 要解释什么。例如，对于分类的概率输出\[0.1, 0.2, 0.7\]，我们想要解释的是0.7。考虑以下公式，其中输出为 `y`，这就是我们想解释的内容。

    y = f(x)

考虑以下公式，其中输出为`y`和`z`。由于我们只能针对一个标量值进行归因，我们必须选择是要解释输出 `y` 还是 `z`。在这个例子中，假设模型是目标检测，y 和 z 分别是边界框和对象分类。您需要选择要解释这两个输出中的哪一个。

    y, z = f(x)

`outputs` 的字典格式为：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输出分配的人类可读的名称。一个常见的例子是“概率”。<br/>
 -  "output_tensor_name"：用于识别要解释的输出层的键/值字段。<br/>
 -  [layer]：要解释的输出层。在单一任务模型中，例如表格回归器，它是模型中的最后（最顶端）一层。
</blockquote>

- `inputs`：贡献输出的特征 -- 它们是如何影响输出的。考虑以下公式，其中 `a` 和 `b` 是特征。我们必须选择哪些特征来解释它们是如何贡献的。假设这个模型用于 A/B 测试，其中 `a` 是预测的数据项，而 `b` 标识模型实例是 A 还是 B。您会选择 `a`（或一些子集）作为特征，而不选择 `b`，因为它不会影响预测结果。

    y = f(a,b)

`inputs` 的最小字典格式为：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输入分配的人类可读的名称。一个常见的例子是“特征”。<br/>
 -  "input_tensor_name"：用于特征归因的输入层的键/值字段。<br/>
 -  [layer]：特征归因的输入层。在单个输入张量模型中，它是模型中的第一个（最底部）层。
</blockquote>

In [ ]:
from google.cloud.aiplatform.explain.metadata.tf.v2 import \
    saved_model_metadata_builder

builder = saved_model_metadata_builder.SavedModelMetadataBuilder(MODEL_DIR)
generated_md = builder.get_metadata()
print(generated_md)

# TMP
try:
    import humps
except:
    !pip install pyhumps
    import humps

generated_md = humps.decamelize(generated_md)

input_metadata = aip.explain.ExplanationMetadata.InputMetadata(
    generated_md["inputs"][serving_input]
)
output_metadata = aip.explain.ExplanationMetadata.OutputMetadata(
    generated_md["outputs"][serving_output]
)
metadata = aip.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，此方法需要以下参数：

- `display_name`：`Model`资源的人类可读名称。
- `artifact`：训练模型构件的Cloud Storage位置。
- `serving_container_image_uri`：用于提供服务的容器映像。
- `sync`：是否异步或同步执行上传。
- `explanation_parameters`：用于配置解释`Model`预测的参数。
- `explanation_metadata`：描述解释`Model`输入和输出的元数据。

如果`upload()`方法是异步运行的，可以随后使用`wait()`方法阻塞等待完成。

In [ ]:
model = aip.Model.upload(
    display_name="boston_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

## 部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的人类可读名称。
- `traffic_split`：在端点上发送到此模型的流量百分比，以字典形式指定一个或多个键值对。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"表示上传的这个模型，100表示流量的100％。
如果端点上存在其他模型，需要分配流量，则使用model_id来指定为{ "0": 百分比，model_id: 百分比，... }，其中model_id是要部署到端点的现有模型的模型ID。百分比必须总和为100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作复制品的加速器数量。
- `starting_replica_count`：最初规划的计算实例数。
- `max_replica_count`：要扩展到的最大计算实例数。在本教程中，只提供一个实例。

In [ ]:
DEPLOYED_NAME = "boston-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

获得测试项目

您将使用数据集的测试（保留）部分中的一个示例作为测试项目。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 使用解释做出预测

现在您的`Model`资源已部署到一个`Endpoint`资源，可以通过向`Endpoint`资源发送预测请求来进行在线解释。

#### 请求

每个实例的格式是：

    [feature_list]

由于explain()方法可以接受多个项目（实例），请将您的单个测试项目发送为一个测试项目列表。

#### 响应

explain()调用的响应是一个带有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个实例的预测。
- `deployed_model_id`：执行预测的已部署`Model`资源的Vertex AI标识符。
- `explanations`：特征归因。

In [ ]:
instances_list = [test_item.tolist()]

prediction = endpoint.explain(instances_list)
print(prediction)

### 理解解释回应

首先，您将查看您的模型预测，并将其与实际值进行比较。

In [ ]:
value = prediction[0][0][0]
print("Predicted Value:", value)

### 检查特征归因

接下来，您将查看此特定示例的特征归因。正面归因值表示特定特征将您的模型预测推高了相应的量，而负面归因值则表示相反。

In [ ]:
from tabulate import tabulate

feature_names = [
    "crim",
    "zn",
    "indus",
    "chas",
    "nox",
    "rm",
    "age",
    "dis",
    "rad",
    "tax",
    "ptratio",
    "b",
    "lstat",
]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, test_item[i], attributions["features"][i]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查您的解释和基线

为了更好地理解您得到的特征归因，您应该将它们与您模型的基线进行比较。在大多数情况下，您的归因值的总和加上基线应该非常接近您模型对每个输入的预测值。还要注意，对于回归模型，AI解释返回的`baseline_score`将对发送到您模型的每个示例都是相同的。对于分类模型，每个类都会有自己的基线。

在这一部分，您将向您的模型发送10个测试示例以进行预测，以便将特征归因与基线进行比较。然后，您将通过`sanity_check_explanations`方法对每个测试示例的归因进行一次身心健康检查。

#### 获取解释

In [ ]:
# Prepare 10 test examples to your model for prediction
instances = []
for i in range(10):
    instances.append(x_test[i].tolist())

response = endpoint.explain(instances)

#### 健全性检查

在以下功能中，您对解释进行健全性检查。

In [ ]:
import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

## 下线模型

当您完成预测后，您可以从`Endpoint`资源中下线模型。这将取消所有计算资源，并停止为已部署的模型计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME